In [1]:
import phrase_entity_extraction as ph

In [2]:
sent = """1010 HWY 59 JUNK WEST""".lower() 

max_address = ph.return_max_address3(ph.seq, sent)
if sent.upper().strip() == max_address.upper().strip():
    print("ALL GOOD!")
else:
    print()
    print("## ************** NOT THE SAME!!!! *********************##\n")
encoding = ph.encode_from_word_list(ph.w(sent))
print("Max Address: ", max_address, "\n\nWORD ARRAY: ", ph.w(sent), "\n\nENCODING: ", encoding)
print('\n\n{},"{}"'.format(sent.upper(), encoding))


## ************** NOT THE SAME!!!! *********************##

Max Address:  1010 59 JUNK WEST 

WORD ARRAY:  ['1010', 'hwy', '59', 'junk', 'west'] 

ENCODING:  [['DIGIT', 'ADR_HEAD'], ['WAY'], ['DIGIT', 'ADR_HEAD'], ['ALPHA'], ['COMMONST', 'DIR']]


1010 HWY 59 JUNK WEST,"[['DIGIT', 'ADR_HEAD'], ['WAY'], ['DIGIT', 'ADR_HEAD'], ['ALPHA'], ['COMMONST', 'DIR']]"


In [3]:
ph.return_best_fit(ph.seq, sent)

[[0, 1, 1, ['DIR', 'SUITE'], '1010'], [2, 5, 3, ['ADDRESS'], '59 junk west']]

In [4]:
ph.seq.look_ahead(ph.encode_from_word_list(ph.w("27th AVE n e".lower()))).get_active_sequences()

[]

In [5]:
arr_cands = ph.get_markers(ph.seq, sent, ['ADDRESS', 'POBOX', 'SUITE', 'DIR'])
arr_cands

[[0, 1, 1, ['DIR', 'SUITE'], '1010'],
 [0, 3, 3, ['DIR', 'ADDRESS'], '1010 hwy 59'],
 [2, 3, 1, ['DIR', 'SUITE'], '59'],
 [2, 4, 2, ['DIR', 'ADDRESS'], '59 junk'],
 [2, 5, 3, ['ADDRESS'], '59 junk west']]

In [6]:
ph.decompose_into_dictionary_words(ph.w(sent), ph.seq, ['ADDRESS', 'SUITE', 'POBOX'])

[1, -1, 3, 2, 3]


([1, -1, 3, 2, 3],
 [['SUITE'], '', ['ADDRESS'], ['ADDRESS'], ['ADDRESS']],
 [['ADDRESS']],
 [(['ADDRESS'], '1010 hwy 59')])

In [7]:
filtered_markers = [marker for marker in arr_cands if not (marker[0] == 0 and marker[2] == 1)]
print(filtered_markers)
arr_cands

[[0, 3, 3, ['DIR', 'ADDRESS'], '1010 hwy 59'], [2, 3, 1, ['DIR', 'SUITE'], '59'], [2, 4, 2, ['DIR', 'ADDRESS'], '59 junk'], [2, 5, 3, ['ADDRESS'], '59 junk west']]


[[0, 1, 1, ['DIR', 'SUITE'], '1010'],
 [0, 3, 3, ['DIR', 'ADDRESS'], '1010 hwy 59'],
 [2, 3, 1, ['DIR', 'SUITE'], '59'],
 [2, 4, 2, ['DIR', 'ADDRESS'], '59 junk'],
 [2, 5, 3, ['ADDRESS'], '59 junk west']]

In [14]:
def entitys_overlap(ent1, ent2):
    if ent1[1] <= ent2[0] or ent2[1] <= ent1[0]:
        return False
    else:
        return True

def get_sorted_entity(_markers, entity):
    entities = [arr for arr in _markers if arr[3][0] == entity]
    entities.sort(key=lambda x: int(x[2]))
    return entities
def get_all_sorted_entity(_markers, axis):
    """axis 0:left, 1:right, 2:length, 3:lst<type>, 4:str"""
    markers.sort(key=lambda x: int(x[axis]))
    return markers

In [118]:
def get_markers(seq, sent, lst_targets):
    """Input is like '123 main str' and returns a list of lists
        RETURNS: a list of list, each list being a candidate and having these values.
            idx_beg, idx_end, length, matches (ADDRESS etc), sequence
        ATTN!!  this lowercases stuff, TODO: Generalize this so it doesn't need lowercasing
    """
    sent = str(sent).lower().strip()
    arr_w = ph.w(sent)
    idx_tail = len(arr_w)
    markers = []

    for idx_beg in range(idx_tail):
        for idx_end in range(idx_beg + 1, idx_tail +1):
            next_values = seq.look_ahead(ph.encode_from_word_list(arr_w[idx_beg:idx_end])).get_next_values()
            matches = list(set(next_values) & set(lst_targets) )
            if matches:
                markers.append([idx_beg, idx_end, idx_end - idx_beg, matches, ' '.join(arr_w[idx_beg:idx_end])])

    return markers
st = "po box 10 1010 hwy 59 suite 100 sam wortington "
markers = get_markers(ph.seq, st, ['_ADDRESS_', '_POBOX_', '_SUITE_', '_DIR_'])
markers

[[0, 3, 3, ['_POBOX_'], 'po box 10'],
 [1, 3, 2, ['_POBOX_'], 'box 10'],
 [2, 3, 1, ['_SUITE_'], '10'],
 [3, 4, 1, ['_SUITE_'], '1010'],
 [3, 6, 3, ['_ADDRESS_'], '1010 hwy 59'],
 [5, 6, 1, ['_SUITE_'], '59'],
 [6, 8, 2, ['_SUITE_'], 'suite 100'],
 [7, 8, 1, ['_SUITE_'], '100'],
 [7, 9, 2, ['_ADDRESS_'], '100 sam'],
 [7, 10, 3, ['_ADDRESS_'], '100 sam wortington']]

In [119]:
### addresses = get_sorted_entity(markers, '_ADDRESS_')
suites = get_sorted_entity(markers, '_SUITE_')
pobox = get_sorted_entity(markers, '_POBOX_')
bearing = get_sorted_entity(markers, '_bearing_')

get_all_sorted_entity(markers, 0)


[[0, 3, 3, ['_POBOX_'], 'po box 10'],
 [1, 3, 2, ['_POBOX_'], 'box 10'],
 [2, 3, 1, ['_SUITE_'], '10'],
 [3, 4, 1, ['_SUITE_'], '1010'],
 [3, 6, 3, ['_ADDRESS_'], '1010 hwy 59'],
 [5, 6, 1, ['_SUITE_'], '59'],
 [6, 8, 2, ['_SUITE_'], 'suite 100'],
 [7, 8, 1, ['_SUITE_'], '100'],
 [7, 9, 2, ['_ADDRESS_'], '100 sam'],
 [7, 10, 3, ['_ADDRESS_'], '100 sam wortington']]

In [120]:
    def book_best_fit(arr_domain, markers):
        
        def is_in_dictionary(markers, start, endplus):
            match_starts = [item for item in markers if item[0] == start]
            match_both = [item for item in match_starts if item[1] == endplus]
            return match_both
        
        last_length = [-1]*len(arr_domain)
        for i in range(len(arr_domain)):
            if is_in_dictionary(markers, 0, i+1):
                last_length[i] = i + 1
            if last_length[i] == -1:
                for j in range(i):
                    if last_length[j] != -1 and is_in_dictionary(markers, j+1, i+1):
                        last_length[i] = i - j
                        break
        decompositions = []
        if last_length[-1] != -1:
            idx = len(arr_domain) - 1
            while idx >= 0:
                decompositions.append(is_in_dictionary(markers,idx + 1 - last_length[idx], idx + 1)[0])
                idx -= last_length[idx]
            decompositions = decompositions[::-1]
        return decompositions

 
markers = get_markers(ph.seq, st, ['_ADDRESS_', '_POBOX_', '_SUITE_', '_DIR_'])
book_best_fit(ph.w(st), markers)

[]

In [135]:
def get_sorted_indexes(markers, which, up=True):
    sorted_list = sorted(list(set([marker[which] for marker in markers])))
    return sorted_list if up else sorted_list[::-1]

markers = get_markers(ph.seq, st, ['_ADDRESS_', '_POBOX_', '_SUITE_', '_DIR_'])
lefts = get_sorted_indexes(markers, 0, up=True)
rights = get_sorted_indexes(markers, 1, up=False)
print(lefts)
print(rights)
final_decomps = []
for left in lefts:
    for right in rights:
        subject = ph.w(st)[left:right]
        if not subject or (right - left) <=1:
            continue
        markers = get_markers(ph.seq, " ".join(subject), ['_ADDRESS_', '_POBOX_', '_SUITE_', '_DIR_'])
        decomp = book_best_fit(subject, markers)
        if decomp:
            #print(left, right, subject,'\n',decomp, '\n')
            final_decomps.append(decomp)

best_decomp = None
max_len = 0
for decomp in final_decomps:  
    if any([['_ADDRESS_'] == marker[3] for marker in decomp]):
        if decomp[-1][1] > max_len:
            max_len = decomp[-1][1]
            best_decomp = decomp
        #print("DECOMP:\n", decomp, decomp[0][0], decomp[-1][1])  # select the largest one here and we're done!
print("AND THE BEST IS\n", best_decomp)

[0, 1, 2, 3, 5, 6, 7]
[10, 9, 8, 6, 4, 3]
AND THE BEST IS
 [[0, 3, 3, ['_POBOX_'], 'po box 10'], [3, 6, 3, ['_ADDRESS_'], '1010 hwy 59'], [6, 8, 2, ['_SUITE_'], 'suite 100']]


This is where I left off.  The last cell has the new best_fit, which will return the longest substring of continuous subunits containing an address,(Well have to dup it for po boxes if there isn't any.